In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from io import BytesIO
from urllib.parse import quote
import re

In [ ]:
def fetch_chords(key, num_pages):
    df_list = []
    key_encoded = quote(key)
    for page in range(1, num_pages + 1):
        base_url = f"https://www.scales-chords.com/showchbykey.php?key={key_encoded}"
        url = f"{base_url}&page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'scaleinfotable'})
        
        # Ignore the header row
        rows = table.find_all('tr')[1:]
        
        for row in rows:
            cells = row.find_all('td')
            if cells:
                first_cell = cells[0]
                
                # Collecting the texts from 'b' elements
                chord_elements = first_cell.find_all('b')
                chord_symbol = chord_elements[0].text if chord_elements and len(chord_elements) > 0 else None
                chord_name = chord_elements[1].text if chord_elements and len(chord_elements) > 1 else None
                chord_family = chord_elements[2].text if chord_elements and len(chord_elements) > 2 else None

                # Collecting the texts from 'a' elements with class 'chlink'
                alternate_symbols_elements = first_cell.find_all('a', class_='chlink')
                alternate_symbols = [elem.text for elem in alternate_symbols_elements] if alternate_symbols_elements else None

                # Append to DataFrame
                df_dict = {
                    'tone': key,
                    'chord': chord_symbol,
                    'chord_name': chord_name,
                    'chord_family': chord_family,
                    'alternate_symbols': alternate_symbols
                }
            df_list.append(df_dict)
    df = pd.DataFrame(df_list)                
    return df

In [ ]:
def call_scales_chords_api(params):
    api_url = 'https://www.scales-chords.com/api/scapi.1.3.php'
    
    response = requests.post(api_url, data=params)
    
    if response.status_code == 200:
        return response.text
    else:
        print(f"Erro: {response.status_code}")

In [ ]:
def download_image(image_url, save_path):
    response = requests.get(image_url)
    if response.status_code == 200:
        img_data = BytesIO(response.content)
        img = Image.open(img_data)
        img.save(save_path)
        print(f"Imagem salva em {save_path}")
    else:
        print(f"Erro ao baixar a imagem: {response.status_code}")

In [ ]:
def download_audio(audio_url, save_path):
    response = requests.get(audio_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Áudio salvo em {save_path}")
    else:
        print(f"Erro ao baixar o áudio: {response.status_code}")

In [ ]:
chords = ['C','C#','D','D#','Db','E','Eb','F','F#','G','G#','Gb','A','A#','Ab','B','Bb']
pages  = [12,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31,31]

all_chords_df = pd.DataFrame()

for chord, num_pages in zip(chords, pages):
    df = fetch_chords(chord, num_pages)
    all_chords_df = pd.concat([all_chords_df, df], ignore_index=True)

In [ ]:
# all_chords_df.to_excel('chords.xlsx', index=False)

In [ ]:
instruments = ['guitar', 'piano']

for instrument in instruments:
    for chord in all_chords_df['Chord'].dropna():
        time.sleep(2)
        params = {
            'chord': chord,
            'output': 'image',
            'instrument': instrument
        }
        response_text = call_scales_chords_api(params)
    
        soup = BeautifulSoup(response_text, 'html.parser')
        link_image = soup.find('img')['src']
        
        safe_chord_name = chord.replace('/', '_').replace('\\', '_').replace('#', 'sharp')
        save_path = f"C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/acordes/img/{instrument}_{safe_chord_name}.jpg"
        download_image(link_image, save_path)

In [ ]:
instruments = ['guitar', 'piano']

for instrument in instruments:
    for chord in all_chords_df['Chord'].dropna():
        time.sleep(2)
        params = {
            'chord': chord,
            'output': 'sound',
            'instrument': instrument
        }

        response_text = call_scales_chords_api(params)
        
        soup = BeautifulSoup(response_text, 'html.parser')
        link_audio = soup.find_all('source')
        audio_url = link_audio[1]['src']
            
        safe_chord_name = chord.replace('/', '_').replace('\\', '_').replace('#', 'sharp')
        save_path = f"C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/acordes/audio/{instrument}_{safe_chord_name}.mp3"
        download_audio(audio_url, save_path)

In [ ]:
params = {
    'chord': 'C',
    'output': 'sound',
    'instrument': 'guitar'
}

soup = BeautifulSoup(call_scales_chords_api(params), 'html.parser')
link_audio = soup.find_all('source')
audio_url = link_audio[1]['src']
save_path = f"C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/acordes/audio/{params['instrument']}_{params['chord']}.mp3"
download_audio(audio_url, save_path)

In [ ]:
all_chords_df = pd.read_excel('chords.xlsx')
all_chords_df

In [ ]:
tunning_guitar = [
    {'tunning_id': "",'guitar_string': 'EADGBE','tunning_description': "standard"},
    {'tunning_id': "?tuning=DADGBE",'guitar_string': 'DADGBE', 'tunning_description': "Drop D"},
    {'tunning_id': "?tuning=DsGsCsFsAsDs",'guitar_string': 'DsGsCsFsAsDs', 'tunning_description': "Half Step Down"},
    {'tunning_id': "?tuning=DGCFAD",'guitar_string': 'DGCFAD', 'tunning_description': "Full Step Down"},
    #{'tunning_id': "?tuning=DADFsAD",'guitar_string': 'DADFsAD', 'tunning_description': "Open D"},
    #{'tunning_id': "?tuning=DGDGBD",'guitar_string': 'DGDGBD', 'tunning_description': "Open G"},
    #{'tunning_id': "?tuning=EAEACsE",'guitar_string': 'EAEACsE', 'tunning_description': "Open A"},
    #{'tunning_id': "?tuning=CEGCEG",'guitar_string': 'CEGCEG', 'tunning_description': "Open C"},
    #{'tunning_id': "?tuning=BFsBFsBDs",'guitar_string': 'BFsBFsBDs', 'tunning_description': "Open B"},
    #{'tunning_id': "?tuning=EBEGsBE",'guitar_string': 'EBEGsBE', 'tunning_description': "Open E"},
    #{'tunning_id': "?tuning=CFCFAF",'guitar_string': 'CFCFAF', 'tunning_description': "Open F"},
    #{'tunning_id': "?tuning=CGDAEB",'guitar_string': 'CGDAEB', 'tunning_description': "All fifths"},
    #{'tunning_id': "?tuning=CGDAEG",'guitar_string': 'CGDAEG', 'tunning_description': "New standard"},
    #{'tunning_id': "?tuning=EADGCF",'guitar_string': 'EADGCF', 'tunning_description': "All fourths"},
    #{'tunning_id': "?tuning=EGsCEGsC",'guitar_string': 'EGsCEGsC', 'tunning_description': "Major thirds on E"}
]

In [ ]:
audio_list = []

file_exists = False

for index, chord in enumerate(all_chords_df['Chord'].dropna()):
    key = chord
    key_encoded = quote(key)
    key_encoded

    for tunning in tunning_guitar:
        base_url = f"https://www.scales-chords.com/chord/guitar/{key_encoded}"
        url = f"{base_url}{tunning['tunning_id']}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        audio = soup.find_all('audio')
        for a in audio:
            audio_id = a.get('id')
            clean_audio_id = re.sub(r'music|fast|strum|slow|mp3', '',audio_id)
            clean_audio_id_name = re.sub(r'music|mp3', '',audio_id)
            audio_url = a.find('source')['src']
            download_audio(f'https://www.scales-chords.com{audio_url}', f"C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_audio/t_{tunning['guitar_string']}_{clean_audio_id_name}.mp3")
            audio_dict = {
                'chord': key,
                'audio_id': audio_id,
                'clean_audio_id': clean_audio_id,
                'audio_url': audio_url,
                'tunning_description': tunning['tunning_description'],
                'guitar_string': tunning['guitar_string'],
                # get the index from all
                'id_chord': index
            }
            audio_list.append(audio_dict)
            temp = pd.DataFrame(audio_dict, index=[0])
            temp.to_csv('audio_list.csv', mode='a', header=not file_exists, index=False)
            file_exists = True


In [ ]:
pd.DataFrame(audio_list)

In [ ]:
image_list = []
for chord in all_chords_df['Chord'].dropna():
    key = chord
    key_encoded = quote(key)
    key_encoded

    for tunning in tunning_guitar:
        time.sleep(1)
        base_url = f"https://www.scales-chords.com/chord/guitar/{key_encoded}"
        url = f"{base_url}{tunning['tunning_id']}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        image = soup.find_all('img')


        for i in image:
            alt = i.get('alt')
            image_url = i['src']
            file_name = image_url.split("/",2)[-1]
            image_id = file_name.replace('-','').replace('.png', '')
            try:
                frets = alt.split('frets:')[1].strip().replace(" ", "")
            except:
                frets = None
            image_dict = {
                'chord': key,
                'image_id': image_id,
                'image_link': image_url,
                'frets': frets
            }
            image_list.append(image_dict)
            download_image(f'https://www.scales-chords.com{image_url}', f"C:/Users/victor.serra/OneDrive - americanas s.a/Área de Trabalho/python/chords/variation_chord_img/t_{tunning['guitar_string']}_{image_id}.png")

df_image_list = pd.DataFrame(image_list)